In [29]:
import urllib.request
from bs4 import BeautifulSoup
import re
import json

base_url = 'https://chicago.legistar.com/'
bill_record_url = 'https://chicago.legistar.com/LegislationDetail.aspx?ID=3481446&GUID=27CD2952-7417-45DD-82F5-02FFA9E03DA7&Options=Advanced&Search='

with urllib.request.urlopen(bill_record_url) as response:
    page = response.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    
    testID = 'ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl06_hypDetails'
    el = soup.select('#'+testID)[0]
    #print(el)
    
    script_string = el['onclick']
    #print(script_string)
    url_postfix = list(filter(None, re.split("[(', ]",script_string)))[1]
    url_string = base_url + url_postfix
    
print(url_string)


https://chicago.legistar.com/HistoryDetail.aspx?ID=15140828&GUID=8EA223C9-BA3D-41F3-A087-62B835565C43


In [37]:
with urllib.request.urlopen(url_string) as response:
    page = response.read()
    soup = BeautifulSoup(page, 'html.parser')
    #print(soup.prettify())
    
    rows = soup.select('.rgRow')
    votes_record = []
    ward_id = 0
    for row in rows:
        name = row.a.text
        vote = row.find_all('td')[1].text
        info_page = base_url + row.a['href']
        votes_record.append({"name":name, "vote":vote, "ward":ward_id, "info_page":info_page})
        
        ward_id += 1 # these records are in order, which is convenient (can verify by checking info page)

print(votes_record)



[{'name': 'Moreno, Proco Joe', 'vote': 'Yea', 'ward': 0, 'info_page': 'https://chicago.legistar.com/PersonDetail.aspx?ID=61031&GUID=5F536F64-389D-40B2-93F4-2F6F877D44E9&Search='}, {'name': 'Dowell, Pat', 'vote': 'Yea', 'ward': 1, 'info_page': 'https://chicago.legistar.com/PersonDetail.aspx?ID=60949&GUID=EECABA2B-D3DD-458C-A7DA-EEE315E4BCBD&Search='}, {'name': 'Hairston, Leslie A.', 'vote': 'Yea', 'ward': 2, 'info_page': 'https://chicago.legistar.com/PersonDetail.aspx?ID=60951&GUID=05B11B79-0F31-4541-8849-553167F02688&Search='}, {'name': 'Mitchell, Gregory I.', 'vote': 'Yea', 'ward': 3, 'info_page': 'https://chicago.legistar.com/PersonDetail.aspx?ID=152685&GUID=28859B53-E49D-4473-B3E5-D4C18E954334&Search='}, {'name': 'Beale, Anthony', 'vote': 'Yea', 'ward': 4, 'info_page': 'https://chicago.legistar.com/PersonDetail.aspx?ID=60955&GUID=970F7168-7484-438F-A713-A90F0AE710E5&Search='}, {'name': 'Thompson, Patrick D.', 'vote': 'Yea', 'ward': 5, 'info_page': 'https://chicago.legistar.com/Perso

In [36]:
# dump record of votes to a json file

target_path = 'sample_council_votes.json'
with open(target_path, 'w') as outfile:
    json.dump(votes_record, outfile)
    
    
